# AI Alignment with the REINFORCE algorithm
Вместо принятого для метода алаймента RLHF использования вычислительно тяжёлого алгоритма Proximal Policy Optimization, считающего генерацию каждого токена моделью за отдельное действие, реализуем алгоритм REINFORCE, воспринимающий за действия агента уже сгенерированные последовательности.
## Level 1
Реализуем REINFORCE w/ baseline, равным moving average.<br>
<b>Сетап.</b> В качестве RewardModel будем использовать ту же модель, что и для самого процесса алаймента $-$ SmolLM2-Instruct на 135M параметров.<br><br>
<b>Датасет.</b> В использующемся [датасете](https://huggingface.co/datasets/esfrankel17/HelpSteer2_binarized) представлены следующие столбцы:<br>

| Prompt | Chosen | Chosen Rating | Rejected | Rejected Rating |
| :-: | :-: | :-: | :-: | :-: |

Согласно документации RewardTrainer, [ссылающейся](https://huggingface.co/docs/trl/main/en/reward_trainer#adding-a-margin-to-the-loss) на Llama 2.1 paper, `Chosen Rating` и `Rejected Rating` можно использовать для вычисления столбца `margin`, который затем учитывается в лоссе RewardModel. Откажемся от этой практики, т.к. она больше не является state-of-the-art стандартом <i>(Llama 3.1 paper, section 4.1.2)</i>.
<br><br>
<b>Приступаем к коду.</b> Импортируем нужные библиотеки, инициализируем модели, загружаем датасет.

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from trl import RewardTrainer, RewardConfig

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
reward_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, use_fast=False)

dataset = load_dataset("esfrankel17/HelpSteer2_binarized")['average_rating_split']

dataset = dataset.remove_columns(["chosen_rating", "rejected_rating"]) # Not SOTA anymore to use 'margin' (Llama 3.1 paper)
dataset[0]

Разделим датасет на тренировочную и валидационную выборки.

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

Начнем тренировку RewardModel с помощью RewardTrainer. Не забудем указать рекомендованное значение для параметра `center_rewards_coefficient`, т.к. хотим, чтобы средний аутпут модели награды был равен 0.

In [ ]:
reward_config = RewardConfig(
    report_to="none",
    learning_rate=5e-5,
    fp16=True,
    max_length=256,
    center_rewards_coefficient = 0.01, # Recommended, as it is preferred that the reward's model output is mean zero.
    output_dir="./reward_model",
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
)

trainer = RewardTrainer(
    model=reward_model,
    args=reward_config,
    processing_class=tokenizer,
    train_dataset=train_dataset.remove_columns("prompt"),
    eval_dataset=val_dataset.remove_columns("prompt"),
)

trainer.train()

Реализуем сам REINFORCE. Формула из оригинальной статьи выглядит следующим образом:<br>
$ E_ {x\sim D,y\sim \pi_\theta (x)} [(R(y,x)-b) \nabla _ {\theta } $ $ \log _ {\pi_\theta }  (y|x)]$<br>
Иными словами, для каждого промпта $x$ мы хотим засэмплировать (согласно вероятностному распределению политики модели $\pi_\theta$) некую респонс-последовательность $y$. Для каждой такой пары для вычисления лосса нам понадобится знать награду $R(x,y)$, вычисленной для данной пары нашей RewardModel, а так же вероятность именно этого $y$ в изначальном распределении. В качестве baseline используем moving average, то есть $b_{MA} = \frac{1}{S} \sum_s R(x^s, y^s)$.<br>
Ключевые моменты процесса тренировки:<br>
* Для оптимизации везде работаем с батчами данных
* Для получения в меру возможностей разумных ответов, каждый раз, передавая промпт в модельку, применяем шаблон чата
* Для вычисления вероятности сэмплирования $y$ - маскируем промпты, для вычисления награды $R(x,y)$ - нет

In [ ]:
from transformers import AutoModelForCausalLM
import torch.optim as optim
import numpy as np

sft_model = AutoModelForCausalLM.from_pretrained(model_name)

#reward_model = AutoModelForSequenceClassification.from_pretrained("checkpoint", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, padding_side="left")
optimizer = optim.Adam(sft_model.parameters(), lr=0.01)
num_epochs = 1
max_length = 50

def get_reward(reward_model, tokenizer, responses):
    rewards = []
    for response in responses:
        inputs = tokenizer.encode_plus(response, truncation=True, padding="max_length", max_length=256, return_tensors="pt")
    
        with torch.no_grad():
            outputs = reward_model(**inputs)
    
        logits = outputs.logits
        rewards.append(logits.item())
    return rewards

def evaluate():
    all_rewards = []
    for batch in val_dataset.iter(batch_size=16):
        prompts = [tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False) for prompt in batch["prompt"]]
        tokenized_prompts = tokenizer.batch_encode_plus(prompts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
        with torch.no_grad():
            outputs = sft_model.generate(input_ids=tokenized_prompts["input_ids"], attention_mask=tokenized_prompts["attention_mask"], max_new_tokens=max_length)
        responses = tokenizer.batch_decode(outputs, skip_special_tokens=False)
        rewards = get_reward(reward_model, tokenizer, responses)
        print("New batch!")
        all_rewards.extend(rewards)
    print(f"Mean reward on the validation dataset: {sum(all_rewards) / len(all_rewards)}")

def train():
    for epoch in range(1, num_epochs + 1):
        all_rewards = []
        for batch in train_dataset.iter(batch_size=16):

            print("New batch!")
            optimizer.zero_grad()
            
            # Tokenize prompts
            prompts = [tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False) for prompt in batch["prompt"]]
            tokenized_prompts = tokenizer.batch_encode_plus(
                prompts, 
                padding="max_length", 
                truncation=True, 
                max_length=max_length, 
                return_tensors="pt"
            ).to(sft_model.device)

            # Generate responses without tracking gradients
            with torch.no_grad():
                generated_sequences = sft_model.generate(
                    input_ids=tokenized_prompts['input_ids'],
                    attention_mask=tokenized_prompts['attention_mask'],
                    max_new_tokens=max_length,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
            print("Generated.")
            # Compute log probabilities for generated tokens
            outputs = sft_model(generated_sequences, return_dict=True)
            logits = outputs.logits
            
            # Calculate log probs for generated tokens (excluding prompt)
            shift_logits = logits[:, :-1, :]  # Skip last token logits
            shift_labels = generated_sequences[:, 1:]  # Skip first token (prompt start)
            
            # Create mask to ignore prompt tokens
            prompt_lengths = tokenized_prompts['attention_mask'].sum(dim=1)
            mask = torch.zeros_like(shift_labels, dtype=torch.bool)
            for i, length in enumerate(prompt_lengths):
                mask[i, length-1:] = True  # Start masking from end of prompt
                
            # Compute log probabilities
            log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
            selected_log_probs = log_probs.gather(dim=-1, index=shift_labels.unsqueeze(-1)).squeeze(-1)
            selected_log_probs = selected_log_probs * mask  # Zero out prompt tokens
            total_log_probs = selected_log_probs.sum(dim=1)

            responses = tokenizer.batch_decode(generated_sequences, skip_special_tokens=False)
            rewards = torch.tensor(get_reward(reward_model, tokenizer, responses)).float().to(sft_model.device)
            
            
            baseline = np.mean(all_rewards) if all_rewards else 0
            rewards = rewards - baseline
            all_rewards.append(rewards.mean().item()) # We can do that as 'mean' is an associative operation
            
            # Compute loss
            loss = (-total_log_probs * rewards).sum()
            
            # Backpropagate
            loss.backward()
            optimizer.step()
            print("Updated the weights")

evaluate()
train()
evaluate()

По итогу тренировки получили весьма значительное для одной эпохи улучшение:

| Mean reward pre-RLHF | Mean reward post-RLHF |
| :-: | :-: |
| -1.59 | -1.11 |

Сохраняем модель, радуемся жизни.

In [ ]:
sft_model.save_pretrained("sft")

## Level 2
Обучим Reward Model на выдачу вероятностного распределения.<br>
Какую информацию мы можем найти в вероятностном распределении оценок? Очевидно, основа такой оценки награды - матожидание вероятностного распределения. С другой стороны, если мы будем опираться лишь на матожидание, то такой подход, во-первых, будет мало отличаться от скалярного значения аутпута модели награды из Level 1, а во-вторых, является очень грубым: когда модель награды не понимает, что сказать, и выдаёт выборку с вероятностью $0,5$ у оценки "$1$" и с вероятностью $0,5$ у оценки "$10$", вряд ли мы захотим, чтобы веса LM оставались примерно такими же. Поэтому для вычисления лосса будем также учитывать дисперсию распределения. Итоговая формула будет выглядеть примерно следующим образом:<br>
$ E_ {x\sim D,y\sim \pi_\theta (x), r \sim p(r|y,x)} [(E_[r]-b-\lambda Var(r)) \nabla _ {\theta } $ $ \log _ {\pi_\theta }  (y|x)]$<br>
Где $\lambda$ - коэффицент того, насколько дисперсия влияет на лосс.<br><br>
<b>Датасет.</b> Преобработаем датасет, превратив `chosen_rating` и `rejected_rating` в вероятностные распределения по следюущему правилу: $2,5$ - середина отрезка $[2, 3]$, превратим в распределение с вероятностью $0,5$ у оценки "$2$" и $0,5$ у оценки "$3$". В исходном датасете значения этих полей находятся на отрезке $[0, 4]$, построим отображение $f : [0, 4] \rightarrow [1, 10]$, используя формулу $\frac{(x-a)(d-c)}{(b-a)}+c$.<br> $f(x) = \frac{9}{4}x + 1$.<br><br>
<b>Приступаем к коду.</b> Начальный этап подготовки и тренировки модели наград остался неизменным с Level 1, за исключением функции предобработки данных.

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import numpy as np

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
reward_distribution_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=10)
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, use_fast=False)

dataset = load_dataset("esfrankel17/HelpSteer2_binarized")['average_rating_split']

def translate(x : float):
    return 2.25*x + 1

def to_distribution(reward : float):
    reward = translate(reward)
    distribution = np.zeros(10)

    rounded = int(reward)
    prob_bigger = reward - rounded
    prob_smaller = 1 - prob_bigger

    distribution[rounded-1] = prob_smaller
    if rounded<=9:
        distribution[rounded] = prob_bigger

    return distribution
    
def tokenize(text, max_length):
    text = tokenizer.apply_chat_template(text, tokenize=False)
    tokenized_text = tokenizer.encode_plus(
        text, 
        padding="max_length", 
        truncation=True, 
        max_length=max_length, 
        return_tensors="pt"
    )
    return tokenized_text
    
def preprocess(examples):
    chosen_tokenized = tokenize(examples["chosen"], 256)
    rejected_tokenized = tokenize(examples["rejected"], 256)

    return {
        "input_ids_chosen": chosen_tokenized["input_ids"].squeeze(0),
        "attention_mask_chosen": chosen_tokenized["attention_mask"].squeeze(0),
        "input_ids_rejected": rejected_tokenized["input_ids"].squeeze(0),
        "attention_mask_rejected": rejected_tokenized["attention_mask"].squeeze(0),
        "chosen_labels": to_distribution(examples["chosen_rating"]),
        "rejected_labels": to_distribution(examples["rejected_rating"]),
    }

dataset = dataset.map(preprocess)

Разбиваем датасет на тренировочную/валидационную выборки.

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

Начинаем тренировку RewardModel.<br>
Очевидно, дефолтный лосс нам тут не подойдет, так как, во-первых, классов у нас не один, а десять (награды 1-10), а во-вторых, работаем мы с логитами. Переопределим RewardTrainer и дефолтный `data_collator`, чтобы в сам лосс к нам обязательно попадали все столбцы, в т.ч. неиспользуемые в forward (в этом конкретном случае - `chosen_labels`, `rejected_labels`).

In [ ]:
from trl import RewardTrainer, RewardConfig
from transformers import DataCollatorWithPadding

class RewardDataCollator(DataCollatorWithPadding):
    def __init__(self, tokenizer):
        super().__init__(tokenizer, padding=True)

    def __call__(self, features):
        # Prepare the inputs for padding
        chosen_features = [{"input_ids": f["input_ids_chosen"], "attention_mask": f["attention_mask_chosen"]} for f in features]
        rejected_features = [{"input_ids": f["input_ids_rejected"], "attention_mask": f["attention_mask_rejected"]} for f in features]

        # Use the parent class's __call__ method to pad inputs
        chosen_batch = super().__call__(chosen_features)
        rejected_batch = super().__call__(rejected_features)

        # Include labels
        batch = {
            "input_ids_chosen": chosen_batch["input_ids"],
            "attention_mask_chosen": chosen_batch["attention_mask"],
            "input_ids_rejected": rejected_batch["input_ids"],
            "attention_mask_rejected": rejected_batch["attention_mask"],
            "chosen_labels": torch.tensor([f["chosen_labels"] for f in features], dtype=torch.float),
            "rejected_labels": torch.tensor([f["rejected_labels"] for f in features], dtype=torch.float),
        }
        return batch

class DistributionRewardTrainer(RewardTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        #print(inputs["input_ids_chosen"])
        # Forward pass for CHOSEN responses
        #print(inputs["input_ids_chosen"].shape)
        chosen_outputs = model(
            input_ids=inputs["input_ids_chosen"],
            attention_mask=inputs["attention_mask_chosen"],
        )
        # Log probabilities for chosen
        chosen_log_probs = torch.log_softmax(chosen_outputs.logits, dim=-1)
        # KL divergence: compares predicted (log_probs) vs target (chosen_labels)
        chosen_loss = torch.nn.functional.kl_div(
            chosen_log_probs,
            inputs["chosen_labels"],
            reduction="batchmean",     # Average loss over the batch
        )

        # Forward pass for REJECTED responses
        rejected_outputs = model(
            input_ids=inputs["input_ids_rejected"],
            attention_mask=inputs["attention_mask_rejected"],
        )
        rejected_log_probs = torch.log_softmax(rejected_outputs.logits, dim=-1)
        rejected_loss = torch.nn.functional.kl_div(
            rejected_log_probs,
            inputs["rejected_labels"],
            reduction="batchmean",
        )

        total_loss = chosen_loss + rejected_loss

        if return_outputs:
            return total_loss, {
                "chosen_outputs": chosen_outputs,
                "rejected_outputs": rejected_outputs
            }
        return total_loss

data_collator = RewardDataCollator(tokenizer)

reward_config = RewardConfig(
    report_to="none",
    learning_rate=5e-5,
    fp16=True,
    max_length=256,
    center_rewards_coefficient = 0.01, # Recommended, as it is preferred that the reward's model output is mean zero.
    output_dir="./reward_model_distribution",
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="no",
    save_strategy="epoch",
)

trainer = DistributionRewardTrainer(
    model=reward_model,
    args=reward_config,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    
)

trainer.train()

Реализуем выведенный раннее подход к вычислению лосса для "распределительной" модели наград. Функцию `evaluate` оставляем старую для возможности сравнения результатов между двумя моделями.

In [ ]:
from transformers import AutoModelForCausalLM
import torch.optim as optim
import numpy as np

sft_model = AutoModelForCausalLM.from_pretrained(model_name)

#reward_model = AutoModelForSequenceClassification.from_pretrained("checkpoint", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, padding_side="left")
optimizer = optim.Adam(sft_model.parameters(), lr=0.01)
num_epochs = 1
max_length = 50

def get_reward_distribution(reward_model, tokenizer, responses):
    rewards = []
    for response in responses:
        inputs = tokenizer.encode_plus(response, truncation=True, padding="max_length", max_length=256, return_tensors="pt")
    
        with torch.no_grad():
            outputs = reward_model(**inputs)
    
        logits = outputs.logits
        log_probs = torch.softmax(logits, dim=-1).squeeze(0).tolist()
        rewards.append(log_probs)
    return rewards

def train():
    lambda_coeff = 0.1 # Check the formula above
    for epoch in range(1, num_epochs + 1):
        all_rewards = []
        for batch in train_dataset.iter(batch_size=16):

            print("New batch!")
            optimizer.zero_grad()
            
            # Tokenize prompts
            prompts = [tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False) for prompt in batch["prompt"]]
            tokenized_prompts = tokenizer.batch_encode_plus(
                prompts, 
                padding="max_length", 
                truncation=True, 
                max_length=max_length, 
                return_tensors="pt"
            ).to(sft_model.device)

            # Generate responses without tracking gradients
            with torch.no_grad():
                generated_sequences = sft_model.generate(
                    input_ids=tokenized_prompts['input_ids'],
                    attention_mask=tokenized_prompts['attention_mask'],
                    max_new_tokens=max_length,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
            print("Generated.")
            # Compute log probabilities for generated tokens
            outputs = sft_model(generated_sequences, return_dict=True)
            logits = outputs.logits
            
            # Calculate log probs for generated tokens (excluding prompt)
            shift_logits = logits[:, :-1, :]  # Skip last token logits
            shift_labels = generated_sequences[:, 1:]  # Skip first token (prompt start)
            
            # Create mask to ignore prompt tokens
            prompt_lengths = tokenized_prompts['attention_mask'].sum(dim=1)
            mask = torch.zeros_like(shift_labels, dtype=torch.bool)
            for i, length in enumerate(prompt_lengths):
                mask[i, length-1:] = True  # Start masking from end of prompt
                
            # Compute log probabilities
            log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
            selected_log_probs = log_probs.gather(dim=-1, index=shift_labels.unsqueeze(-1)).squeeze(-1)
            selected_log_probs = selected_log_probs * mask  # Zero out prompt tokens
            total_log_probs = selected_log_probs.sum(dim=1)

            responses = tokenizer.batch_decode(generated_sequences, skip_special_tokens=False)

            baseline = np.mean(all_rewards) if all_rewards else 0
            
            rewards = get_reward_distribution(reward_distribution_model, tokenizer, responses)
            
            expected_values = [sum([distr[x]*(x+1) for x in range(10)]) for distr in rewards]
            expected_values_sq = [sum([distr[x]*(x+1)**2 for x in range(10)]) for distr in rewards]
            most_probable = [np.argmax(distr)+1 for distr in rewards]
            var = [expected_values_sq[i] - expected_values[i]**2 for i in range(len(rewards))]
            
            expected_with_variance = [expected_values[i] - baseline - var[i]*lambda_coeff]
            
            all_rewards.extend(most_probable)
            
            # Compute loss
            loss = (-total_log_probs * np.sum(expected_with_variance)).sum()
            
            # Backpropagate
            loss.backward()
            optimizer.step()
            print("Updated the weights")

evaluate()
train()
evaluate()

Т.к. функция `evaluate` работает с предыдущей RewardModel, выдающей скалярные значения, можем получить более-менее сравнимые данные:
| Mean reward pre-RLHF | Mean reward post-RLHF |
| :-: | :-: |
| -2.02 | -1.23 |

Что видим в итоге? REINFORCE со "скалярной" RewardModel улучшил значение средней награды на валидационном датасете на ~30%, REINFORCE с "распределительной" - на ~40%. В сравнении абсолютных значений последняя модель тоже отрывается.

## Выводы
Почему так получилось? Думаю, что своим отрывом "распределительная" RewardModel в алгоритме REINFORCE обязана дополнительному параметру "неуверенности" модели - дисперсии. Мы не имели доступа к таким данным со "скалярной" моделью награды; как выяснилось, достаточно часто модель мечется между несколькими, казалось бы, противоположными классами-оценками награды. 
### Что крутого узналось?
* Изначально я не планировал использовать chat template на промпте/респонсе модели, но, на удивление, это ОЧЕНЬ сильно повысило "разумность" ответов модели. Конечно, в ходе тренировки модель, в токенизированном входе/выходе которой не учавствует шаблон чата, рано или поздно нашла бы способ найти некий минимум функции лосса, "обманывая" модель награды; но что это тогда за RLHF? Так что несмотря на несколько усложненный процесс токенизации, оно того однозначно стоило.
* Много времени кодинга было убито на то, чтобы разрешить следующую проблему: первоначально, в коде для получения вероятности сэмплирования последовательности $y$ был использован многократный `model(**inputs)` (для каждого нового токена) с вычислением градиента. Это более интуитивный подход, вычисляющий вероятность сэмплирования каждого токена и затем уже всей последовательности. Но, как оказалось, в слегка контритуитивном переходе на `with torch.no_grad(): model.generate(**inputs)` (то есть генерации сразу всей последовательности и вычисления вероятности ее сэмплирования пост-фактум) кроилось решение оптимизации вычислений во много-много раз.
### Что не получилось реализовать?
В силу временных и вычислительных ограничений у меня не вышло сохранить один валидационный датасет на обе модели награды, "скалярную" и "распределительную"; более того, последняя обучилась на слегка мЕньшем срезе данных из датасета. Оба фактора могут повлиять на воспроизводимость полученных результатов.